<a href="https://colab.research.google.com/github/eespejoruiz/IndexNow/blob/main/Index_Now_%7C_Expreso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) API Search Console Index Now


In [ ]:
!pip3 install git+https://github.com/antoineeripret/gsc_wrapper

# 2) Instalo Librerías

In [ ]:
import gscwrapper
import pandas as pd
import numpy as np
from gscwrapper.account import Account

In [ ]:
!pip show gscwrapper

# 3) Me conecto a google drive


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

# 4) Me Conecto al API y Selecciono de Propiedad en Google Search Console


In [ ]:
account_service = (
    gscwrapper
    .generate_auth(
        client_config="/content/drive/MyDrive/000 _ Expreso/search-console-450718-997bc7dc75d1.json",
        service_account_auth=True
    )
)

In [ ]:
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build

# 📌 Ruta del archivo JSON de la cuenta de servicio en Google Drive
credenciales_json = "/content/drive/MyDrive/000 _ Expreso/indexing cs.json"

# 📌 Definir el `scope` correcto para la API de Indexing
SCOPES = ["https://www.googleapis.com/auth/indexing"]

def autenticar_con_cuenta_de_servicio():
    """Autentica con una cuenta de servicio sin intervención manual."""

    creds = service_account.Credentials.from_service_account_file(
        credenciales_json,
        scopes=SCOPES
    )

    return creds

# 🔹 Ejecutar la autenticación con la cuenta de servicio
credentials = autenticar_con_cuenta_de_servicio()

# 🔹 Crear servicio para la API de Indexing con los permisos correctos
service = build("indexing", "v3", credentials=credentials)

print("✅ Autenticación exitosa con cuenta de servicio. 'service' está listo para Indexing API.")

In [ ]:
account_service

# 5) Extraer URLs del Sitemap

In [ ]:
import requests
import xml.etree.ElementTree as ET
import os
import gzip
import io
import time

# 📌 URLs base
sitemap_index_url = "https://www.expreso.ec/sitemap-index.xml"
ruta_indexadas = "/content/drive/MyDrive/000 _ Expreso/urls_indexadas.txt"
ruta_por_indexar = "/content/drive/MyDrive/000 _ Expreso/urls_por_indexar.txt"

def obtener_sitemaps(url):
    """Obtiene la lista de archivos sitemap desde el sitemap index con reintentos en caso de fallo."""
    for intento in range(3):  # Reintentar hasta 3 veces
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            # Verificar si el contenido es XML válido
            if "xml" in response.headers.get("Content-Type", ""):
                root = ET.fromstring(response.content)
                sitemaps = [elem.text for elem in root.findall(".//{http://www.sitemaps.org/schemas/sitemap/0.9}loc")]
                return sitemaps
            else:
                print("❌ El contenido recibido no es XML válido.")
                return []

        except requests.exceptions.RequestException as e:
            print(f"⚠️ Error al obtener el sitemap index. Intento {intento + 1}/3: {e}")
            time.sleep(2)

    print("❌ No se pudo obtener el sitemap index después de 3 intentos.")
    return []

def obtener_urls_de_sitemaps(lista_sitemaps):
    """Escanea todos los sitemaps y extrae las URLs, descomprimiendo archivos .gz si es necesario."""
    urls_totales = set()

    for sitemap in lista_sitemaps:
        for intento in range(3):  # Reintentar hasta 3 veces en caso de error
            try:
                response = requests.get(sitemap, timeout=10)
                response.raise_for_status()

                # Si el archivo es .xml.gz, descomprimirlo antes de procesarlo
                if sitemap.endswith(".gz"):
                    with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as f:
                        xml_content = f.read().decode("utf-8")
                else:
                    xml_content = response.content.decode("utf-8")

                # Validar si el XML es válido
                if not xml_content.strip().startswith("<"):
                    print(f"⚠️ {sitemap} no contiene XML válido.")
                    continue

                # Parsear el XML
                root = ET.fromstring(xml_content)
                urls = [elem.text for elem in root.findall(".//{http://www.sitemaps.org/schemas/sitemap/0.9}loc")]
                urls_totales.update(urls)
                break  # Salir del bucle si el intento fue exitoso

            except (requests.exceptions.RequestException, ET.ParseError) as e:
                print(f"⚠️ Error en intento {intento + 1}/3 con {sitemap}: {e}")
                time.sleep(2)  # Esperar antes de reintentar

    return urls_totales

def leer_urls_indexadas(ruta):
    """Lee las URLs ya indexadas desde el archivo."""
    if os.path.exists(ruta):
        with open(ruta, "r") as f:
            return set(f.read().splitlines())
    return set()

def guardar_urls_nuevas(ruta, urls_nuevas):
    """Guarda solo las nuevas URLs en el archivo (sobrescribe el archivo anterior)."""
    if urls_nuevas:
        with open(ruta, "w") as f:  # ⚠️ 'w' para sobrescribir el archivo anterior
            for url in urls_nuevas:
                f.write(url + "\n")
        print(f"✅ {len(urls_nuevas)} nuevas URLs guardadas en '{ruta}'.")
    else:
        print("⚠️ No hay nuevas URLs para agregar.")

def actualizar_urls_indexadas(ruta, urls_nuevas):
    """Agrega las nuevas URLs indexadas al archivo `urls_indexadas.txt`."""
    if urls_nuevas:
        with open(ruta, "a") as f:  # ⚠️ 'a' para agregar nuevas URLs sin borrar las anteriores
            for url in urls_nuevas:
                f.write(url + "\n")
        print(f"✅ {len(urls_nuevas)} URLs añadidas a '{ruta}' para futuras referencias.")

# 🔹 1️⃣ Obtener todos los archivos de sitemap desde el sitemap index
sitemaps_encontrados = obtener_sitemaps(sitemap_index_url)
print(f"🔍 Se encontraron {len(sitemaps_encontrados)} sitemaps.")

# 🔹 2️⃣ Extraer todas las URLs de los sitemaps
urls_extraidas = obtener_urls_de_sitemaps(sitemaps_encontrados)
print(f"✅ Total de URLs extraídas: {len(urls_extraidas)}")

# 🔹 3️⃣ Leer el archivo de URLs indexadas y filtrar las nuevas
urls_indexadas = leer_urls_indexadas(ruta_indexadas)
urls_nuevas = urls_extraidas - urls_indexadas

# 🔹 4️⃣ Guardar solo las nuevas URLs en `urls_por_indexar.txt` para ser enviadas
guardar_urls_nuevas(ruta_por_indexar, urls_nuevas)

# 🔹 5️⃣ Actualizar `urls_indexadas.txt` con las nuevas URLs enviadas
actualizar_urls_indexadas(ruta_indexadas, urls_nuevas)

# 6) Enviar las URLs Nuevas a Google Search Console

In [ ]:
import time
import os

# 📌 Ruta del archivo donde están las URLs a indexar
ruta_urls = "/content/drive/MyDrive/000 _ Expreso/urls_por_indexar.txt"
ruta_errores = "/content/drive/MyDrive/000 _ Expreso/urls_fallidas.txt"

def leer_urls(ruta):
    """Lee las URLs desde un archivo y las devuelve como una lista."""
    if os.path.exists(ruta):
        with open(ruta, "r") as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    return []

def enviar_url_para_indexacion(url):
    """Envía una URL a la API de Indexing de Google y muestra el estado del envío."""
    request_body = {
        "url": url,
        "type": "URL_UPDATED"
    }

    try:
        response = service.urlNotifications().publish(body=request_body).execute()

        # 📌 Verificar si la API respondió correctamente
        if "urlNotificationMetadata" in response:
            print(f"✅ URL enviada con éxito: {url}")
            print(f"🔹 Última actualización: {response['urlNotificationMetadata'].get('latestUpdate', 'No disponible')}\n")
        else:
            print(f"⚠️ URL enviada pero sin confirmación clara: {url}")
            print(f"🔹 Respuesta: {response}\n")

        return True  # Indica que la URL se envió correctamente

    except Exception as e:
        print(f"❌ Error al enviar {url}: {e}\n")
        return False  # Indica que hubo un error

def guardar_urls_fallidas(urls):
    """Guarda las URLs que no pudieron enviarse para reintentos futuros."""
    if urls:
        with open(ruta_errores, "w") as f:
            for url in urls:
                f.write(url + "\n")
        print(f"⚠️ {len(urls)} URLs con error guardadas en '{ruta_errores}'.")

# 🔹 Leer las URLs desde el archivo
urls_a_indexar = leer_urls(ruta_urls)

if not urls_a_indexar:
    print("⚠️ No hay URLs en el archivo para indexar.")
else:
    print(f"🚀 Enviando {len(urls_a_indexar)} URLs a Google Indexing API...")

    # 🔹 Lista de URLs que fallaron al enviarse
    urls_fallidas = []

    # 🔹 Enviar todas las URLs a Google Indexing API
    for url in urls_a_indexar:
        if not enviar_url_para_indexacion(url):
            urls_fallidas.append(url)

        time.sleep(2)  # 🔹 Pausa para evitar bloqueos

    # 🔹 Guardar las URLs con errores para reintento
    guardar_urls_fallidas(urls_fallidas)

    print("✅ Proceso de envío completado.")

# 🔺 Enviar las TODAS las URLs a Google Search Console | Usar solo en casos de cambios generales en la web.

In [ ]:
import os
import time

# Archivos de URLs
ruta_urls = "/content/drive/MyDrive/000 _ Expreso/urls_indexadas.txt"
ruta_errores = "/content/drive/MyDrive/000 _ Expreso/urls_fallidas.txt"

def leer_urls(ruta):
    """Lee las URLs desde un archivo."""
    if os.path.exists(ruta):
        with open(ruta, "r") as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    return []

def guardar_urls_fallidas(urls):
    """Guarda las URLs que fallaron en un archivo para reintentos futuros."""
    if urls:
        with open(ruta_errores, "a") as f:
            for url in urls:
                f.write(url + "\n")
        print(f"⚠️ {len(urls)} URLs con error guardadas en '{ruta_errores}'.")

def enviar_url_para_indexacion(url):
    """Envía una URL a la API de Indexing de Google."""
    request_body = {
        "url": url,
        "type": "URL_UPDATED"
    }
    try:
        response = service.urlNotifications().publish(body=request_body).execute()
        print(f"✅ URL enviada: {url} - Respuesta: {response}")
        return True
    except Exception as e:
        print(f"❌ Error al enviar {url}: {e}")
        return False

# Leer todas las URLs desde el archivo
urls_a_indexar = leer_urls(ruta_urls)

# Filtrar para enviar solo 2000 por día
urls_a_indexar = urls_a_indexar[:2000]

# Lista de URLs que no pudieron enviarse
urls_fallidas = []

if urls_a_indexar:
    print(f"🔹 Enviando {len(urls_a_indexar)} URLs a Google Indexing API...")
    for url in urls_a_indexar:
        if not enviar_url_para_indexacion(url):
            urls_fallidas.append(url)
        time.sleep(2)  # Pausa para evitar bloqueos
else:
    print("⚠️ No hay URLs en el archivo para indexar.")

# Guardar las URLs con errores
guardar_urls_fallidas(urls_fallidas)

# 📌 Código: Reintentar el envío de URLs fallidas

In [ ]:
import os
import time

# Archivo con las URLs que fallaron en intentos anteriores
ruta_errores = "/content/drive/MyDrive/000 _ Expreso/urls_fallidas.txt"

def leer_urls(ruta):
    """Lee las URLs desde un archivo."""
    if os.path.exists(ruta):
        with open(ruta, "r") as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    return []

def guardar_urls_fallidas(urls):
    """Sobrescribe el archivo con las URLs que aún fallan."""
    with open(ruta_errores, "w") as f:
        for url in urls:
            f.write(url + "\n")
    print(f"⚠️ {len(urls)} URLs siguen fallando y se guardaron en '{ruta_errores}'.")

def enviar_url_para_indexacion(url):
    """Envía una URL a la API de Indexing de Google."""
    request_body = {
        "url": url,
        "type": "URL_UPDATED"
    }
    try:
        response = service.urlNotifications().publish(body=request_body).execute()
        print(f"✅ URL enviada: {url} - Respuesta: {response}")
        return True
    except Exception as e:
        print(f"❌ Error al enviar {url}: {e}")
        return False

# Leer las URLs fallidas
urls_fallidas = leer_urls(ruta_errores)

# Lista de URLs que sigan fallando después del reintento
urls_errores_nuevos = []

if urls_fallidas:
    print(f"🔹 Reintentando enviar {len(urls_fallidas)} URLs a Google Indexing API...")
    for url in urls_fallidas:
        if not enviar_url_para_indexacion(url):
            urls_errores_nuevos.append(url)  # Guardar solo las que siguen fallando
        time.sleep(2)  # Pausa para evitar bloqueos
else:
    print("⚠️ No hay URLs fallidas para reintentar.")

# Guardar las URLs que aún fallan
guardar_urls_fallidas(urls_errores_nuevos)

# 🟢 Verificar y Enviar a Indexar las URLs No Indexadas

In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client requests

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# 📌 Ruta del archivo JSON de la cuenta de servicio en Google Drive
credenciales_json = "/content/drive/MyDrive/000 | Expreso/indexing cs.json"

# 📌 Definir los permisos (scopes) correctos
SCOPES_GSC = ["https://www.googleapis.com/auth/webmasters.readonly"]
SCOPES_INDEXING = ["https://www.googleapis.com/auth/indexing"]

# 🔹 Autenticación con Google Search Console API
credentials_gsc = service_account.Credentials.from_service_account_file(credenciales_json, scopes=SCOPES_GSC)
service_gsc = build("searchconsole", "v1", credentials=credentials_gsc)

# 🔹 Autenticación con Google Indexing API
credentials_indexing = service_account.Credentials.from_service_account_file(credenciales_json, scopes=SCOPES_INDEXING)
service_indexing = build("indexing", "v3", credentials=credentials_indexing)

print("✅ Autenticación exitosa con Google Search Console API y Google Indexing API")

In [ ]:
from googleapiclient.discovery import build
from google.oauth2 import service_account

# 📌 Ruta del archivo JSON de la cuenta de servicio en Google Drive
credenciales_json = "/content/drive/MyDrive/000 _ Expreso/indexing cs.json"

# 📌 Definir los permisos (scopes) correctos
SCOPES = ["https://www.googleapis.com/auth/webmasters.readonly"]

# 🔹 Autenticar con Google Search Console API
credentials = service_account.Credentials.from_service_account_file(credenciales_json, scopes=SCOPES)
service_gsc = build("searchconsole", "v1", credentials=credentials)

# 🔹 Obtener la lista de sitios que la cuenta de servicio puede acceder
sites_list = service_gsc.sites().list().execute()

# 🔹 Mostrar los sitios a los que tiene acceso
for site in sites_list.get("siteEntry", []):
    print(f"🔹 Acceso a: {site['siteUrl']} - Rol: {site['permissionLevel']}")

In [ ]:
import requests
import csv
import time
import gzip
import xml.etree.ElementTree as ET
from io import BytesIO

# 📌 URL del sitemap indexado
sitemap_index_url = "https://www.expreso.ec/sitemap-index.xml"

# 📌 Archivos de salida
output_csv = "/content/drive/MyDrive/000 _ Expreso/resultados_indexacion.csv"
urls_no_indexadas_txt = "/content/drive/MyDrive/000 _ Expreso/urls_no_indexadas.txt"

# 🔹 1️⃣ Obtener todos los sitemaps desde `sitemap-index.xml`
def obtener_sitemaps(url_sitemap_index):
    response = requests.get(url_sitemap_index)
    sitemaps = []

    if response.status_code == 200:
        try:
            root = ET.fromstring(response.content)
            for elem in root.findall(".//{http://www.sitemaps.org/schemas/sitemap/0.9}loc"):
                sitemaps.append(elem.text)
        except ET.ParseError:
            print("❌ Error al analizar el sitemap indexado. Verifica que sea XML válido.")
    else:
        print("❌ Error al obtener el sitemap indexado")

    return sitemaps

# 🔹 2️⃣ Obtener todas las URLs de cada sitemap (maneja archivos .xml y .xml.gz)
def obtener_urls_de_sitemaps(lista_sitemaps):
    urls_totales = []

    for sitemap in lista_sitemaps:
        response = requests.get(sitemap)

        if response.status_code == 200:
            try:
                if sitemap.endswith(".gz"):  # 📌 Si es un archivo comprimido, lo descomprimimos
                    with gzip.GzipFile(fileobj=BytesIO(response.content)) as f:
                        content = f.read()
                else:
                    content = response.content

                root = ET.fromstring(content)
                for elem in root.findall(".//{http://www.sitemaps.org/schemas/sitemap/0.9}loc"):
                    urls_totales.append(elem.text)

            except ET.ParseError:
                print(f"⚠️ Error al analizar {sitemap}. Verifica que sea XML válido.")

        else:
            print(f"❌ Error al obtener {sitemap}")

    return urls_totales

# 🔹 3️⃣ Consultar si una URL está indexada en Google Search Console
def verificar_indexacion(service_gsc, url, site_url):
    try:
        request = {
            "startDate": "2024-01-01",
            "endDate": "2025-02-18",
            "dimensions": ["page"],
            "dimensionFilterGroups": [{
                "filters": [{
                    "dimension": "page",
                    "operator": "equals",
                    "expression": url
                }]
            }],
            "rowLimit": 1
        }

        # 🔹 Consultar la API de GSC
        response = service_gsc.searchanalytics().query(siteUrl=site_url, body=request).execute()

        if "rows" in response and len(response["rows"]) > 0:
            return "✅ Indexada"
        else:
            return "❌ No indexada"

    except Exception as e:
        print(f"⚠️ Error con {url}: {e}")
        return "⚠️ Error"

# 🔹 4️⃣ Ejecutar la verificación de indexación
def analizar_indexacion():
    site_url = "sc-domain:expreso.ec"  # ⚠️ Usa la versión exacta registrada en GSC
    print(f"🔹 Obteniendo lista de sitemaps desde: {sitemap_index_url}")
    lista_sitemaps = obtener_sitemaps(sitemap_index_url)

    urls_sitemap = obtener_urls_de_sitemaps(lista_sitemaps)
    resultados = []
    urls_no_indexadas = []

    print(f"🔹 Verificando indexación de {len(urls_sitemap)} URLs en Google Search Console...")

    for url in urls_sitemap:
        estado = verificar_indexacion(service_gsc, url, site_url)
        resultados.append([url, estado])

        if estado == "❌ No indexada":
            urls_no_indexadas.append(url)

        time.sleep(2)  # 🔹 Pausa para evitar límites de la API

    # Guardar los resultados
    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["URL", "Estado de Indexación"])
        writer.writerows(resultados)

    with open(urls_no_indexadas_txt, "w") as f:
        for url in urls_no_indexadas:
            f.write(url + "\n")

    print("✅ Análisis de indexación completado.")

# 🔹 Ejecutar la verificación de indexación
analizar_indexacion()

In [ ]:
from googleapiclient.discovery import build
from google.oauth2 import service_account

# 📌 Ruta del archivo JSON de la cuenta de servicio en Google Drive
credenciales_json = "/content/drive/MyDrive/000 _ Expreso/indexing cs.json"

# 📌 Definir el permiso (scope) correcto
SCOPES = ["https://www.googleapis.com/auth/webmasters.readonly"]

# 🔹 Autenticar con Google Search Console API
credentials = service_account.Credentials.from_service_account_file(credenciales_json, scopes=SCOPES)
service_gsc = build("searchconsole", "v1", credentials=credentials)

# 🔹 Obtener la lista de sitios que la cuenta de servicio puede acceder
sites_list = service_gsc.sites().list().execute()

# 🔹 Mostrar los sitios a los que tiene acceso
print("🔹 La cuenta de servicio tiene acceso a:")
for site in sites_list.get("siteEntry", []):
    print(f" - {site['siteUrl']} | Permiso: {site['permissionLevel']}")

In [ ]:
import os
import time

# 📌 Archivos de entrada/salida
ruta_urls_no_indexadas = "/content/drive/MyDrive/000 _ Expreso/urls_no_indexadas.txt"
ruta_errores = "/content/drive/MyDrive/000 _ Expreso/urls_fallidas.txt"

def leer_urls_no_indexadas(ruta):
    """Lee las URLs no indexadas desde un archivo."""
    if os.path.exists(ruta):
        with open(ruta, "r") as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    return []

def guardar_urls_fallidas(urls):
    """Guarda las URLs que fallaron en un archivo para reintentos futuros."""
    if urls:
        with open(ruta_errores, "w") as f:
            for url in urls:
                f.write(url + "\n")
        print(f"⚠️ {len(urls)} URLs con error guardadas en '{ruta_errores}'.")

def actualizar_archivo_urls(urls_exitosas):
    """Elimina del archivo las URLs que ya fueron indexadas correctamente."""
    if not os.path.exists(ruta_urls_no_indexadas):
        return

    urls_actuales = leer_urls_no_indexadas(ruta_urls_no_indexadas)
    nuevas_urls = list(set(urls_actuales) - set(urls_exitosas))  # 🔹 Filtrar URLs exitosas

    with open(ruta_urls_no_indexadas, "w") as f:
        for url in nuevas_urls:
            f.write(url + "\n")

    print(f"📄 Archivo actualizado: {len(nuevas_urls)} URLs restantes en '{ruta_urls_no_indexadas}'.")

def enviar_url_para_indexacion(url):
    """Envía una URL a la API de Indexing de Google."""
    request_body = {
        "url": url,
        "type": "URL_UPDATED"
    }
    try:
        response = service_indexing.urlNotifications().publish(body=request_body).execute()
        print(f"✅ URL enviada a indexar: {url}")
        return True
    except Exception as e:
        print(f"❌ Error al enviar {url}: {e}")
        return False

# 🔹 Leer URLs a indexar
urls_a_indexar = leer_urls_no_indexadas(ruta_urls_no_indexadas)
urls_fallidas = []
urls_indexadas = []

if urls_a_indexar:
    print(f"🔹 Enviando {len(urls_a_indexar)} URLs a Google Indexing API...")
    for url in urls_a_indexar:
        if enviar_url_para_indexacion(url):
            urls_indexadas.append(url)  # Guardamos las exitosas
        else:
            urls_fallidas.append(url)  # Guardamos las fallidas
        time.sleep(2)  # 🔹 Evitar bloqueos con la API

# 🔹 Actualizar archivos
guardar_urls_fallidas(urls_fallidas)
actualizar_archivo_urls(urls_indexadas)